# Tunable Ribosome Occupancy Model
_Author: Raghav Chanchani_

_For the Zid Lab at UC San Diego_

_Date last modified: 11/11/2018_
#### A discrete-time single probability chain to simulate ribosome distribution along a given gene using the assumptions listed below. There is no final absorption state. Ribosomes are recycled to the -1 index after they move off of the simulated mRNAs. This model does not consider binding energies of the ribosomes to sequences in the gene of interest. Time is simulated to 0.01sec. precision.
__References:__
1. https://book.bionumbers.org/what-is-faster-transcription-or-translation
2. https://www.columbia.edu/~ks20/stochastic-I/stochastic-I-MCI.pdf
3. https://github.com/gvanderheide/discreteMarkovChain

__Assumptions:__
1. Initiation when ribosome reads first codon _(AUG in given .txt or .fasta gene file)_
2. Elongation in-between
3. Ribosome moves one triplet with each elongation step
4. Ribosomes are recycled once they move off of the gene
5. If any pair of ribosomes' positions are under appropriate distance apart, then the ribosome closer to AUG is not allowed to move until its position $\geq$ the distance threshold from the next ribosome
6. Termination when ribosome reads UAG, UAA, or UGA _(in next update)_
7. Length of gene in graph and calculations is in codons, not nucleotides
8. Position refers to the center position of the ribosome
9. Each second is divided into 100ths to allow for more accurate reflection of kI and kE values
10. Calculations of $\text{time res.}\over{kI\text{ or }kE}$ are "floored" (e.g. math.floor($100\over{kI}$)) in determining when to attempt initiation and elongation

__Tunable Parameters:__
1. Initiation rate
2. Elongation rate
3. Number of ribosomes on a single mRNA
4. Number of mRNAs
5. Probability of a ribosome moving from its current position
6. Size of a ribosome (in nucleotides)
7. How long the simulated run is for (sec.)
8. Time until initiation rate becomes 0 _(occurs stepwise)_
9. Time until elongation rate becomes 0 _(occurs stepwise)_

In [1]:
%matplotlib notebook
import numpy as np
import math
import sys
import os
from os import path
import argparse
import matplotlib.pyplot as plt
from ipywidgets import *
import ipywidgets as widgets
from IPython.display import display
plt.style.use('ggplot')
#from discreteMarkovChain import markovChain

In [2]:
"""
Read in the file containing the gene of interest and determine whether or not it is the correct file type ".txt" and
initialize the mRNA, occupancy, and ribosome_list lists.
"""
global gene_length
global mRNA
global fname
global res
global codon_size

parser = argparse.ArgumentParser()
parser.add_argument('filename')
args = parser.parse_args('Pab1.txt'.split())
ribosome_list = [] # storage for all ribosomes in a single mRNA
gene_length = 0
res = 100 # time resolution (the number of divisions made of a second)
codon_size = 3
mRNA = []
occupancy = [] # storage for all ribosomes in all mRNAs
fname = args.filename
if not fname.lower().endswith(('.txt')):
    parser.error("Input file must be a .txt file")

In [3]:
"""
Description: Creates the ribosome object which stores the ribosome's position and how long
    it has been at its current position. The counter is initially set to zero.
"""
class ribosome:
    """
    Description: Creates a new ribosome object with initial position -1 and counter 0.
    Inputs: None
    Return: None
    """
    def __init__(self):
        self.position = -1
        self.counter = 0
    """
    Description: Changes the value of the ribosome's position.
    Inputs: new_position - the position the ribosome has been moved to
    Return: None
    """
    def set_position(self, new_position):
        self.position = new_position
    """
    Description: Changes the value of the ribosome object's counter variable given a new value.
    Inputs: new_count - number of timesteps ribosome has been at its current position
    Return: None
    """
    def set_counter(self, new_count):
        self.counter = new_count

In [4]:
"""
Description: Counts number of nucleotides in the gene of interest and creates an "mRNA" list of the same size
    that will be used to note ribosome positions on.
Inputs: gene_file - a .txt file that contains the mRNA base pairs of a given gene.
Return: gene_length - the length of the gene passed in, in nucleotides
        mRNA - a list corresponding with the length of the (gene_file in codons)
"""
def read_gene(gene_file):
    global gene_length
    global mRNA
    global codon_size
    global target
    temp_list = []
    mRNA = []

    try:
        with open(gene_file) as inputFileHandle:
            line_list = [lines.split() for lines in inputFileHandle]  # extract lines
            while line_list:
                temp_list.extend(line_list.pop(0))
                while temp_list:
                    if ('>') in temp_list[0]:
                        target = temp_list[0].strip('>')
                    mRNA.extend(temp_list.pop(0))
            gene_length = int(len(mRNA)/codon_size)
            return inputFileHandle.read(), gene_length, mRNA
    except IOError:
        sys.stderr.write("read_gene - Error: Could not open {}\n".format(gene_file))
        sys.exit(-1)


In [5]:
"""
Description: Creates a steady state probability matrix that determines the likelihood that a ribosome
    will move to the next codon or remain in its current position.
Inputs: None
Return: None
"""
def make_matrix():
    global probability
    global gene_length
    
    probs = np.zeros[(gene_length,gene_length)]
    for col in range(gene_length):
        for row in range(gene_length):
            if row == col:
                probs[row][col] = probability
                if row == gene_length - 1:
                    probs[row][0] = abs(1-probability)
            if row + 1 == col:
                probs[row][col] = abs(1-probability)
    mc = markovChain(probs)
    # computation method of steady-state probabilities
    mc.computePi('linear')
    # steady-state probabilities of moving into different nucleotides
    steady_state_probs = mc.pi
    print('equilibrium state: {}'.format(steady_state_probs))

    return

In [6]:
"""
Description: Given the probability that the ribosome will move, the ribosome is evaluated to move
    or remain in its current position.
Inputs: ribosome - ribosome object trying to move
Return: ribosome - same ribosome object with updated position and counter
"""
def move(ribosome):
    global gene_length
    global probability
    
    position = ribosome.position
    if ribosome.position <= gene_length:
        moves = np.random.random() <= probability
        if moves:
            ribosome.position = position + 1
            ribosome.counter = 1
        else:
            ribosome.counter += 1
    return ribosome

In [7]:
"""
Description: Graphs the ribosome occupancy of codons as a histogram with x-axis being the position (in codons)
    and the y-axis being occupancy as a fraction of the total number of ribosomes simulated across all mRNA
Inputs: ribos - the list containing all ribosomes
Return: None
"""
def create_histogram(all_cules):
    global fname
    global gene_length
    global res
    global target
    
    pos_array = [ribo.position for sim in all_cules for ribo in sim]
    ax = plt.gca()
    ax.set_title('Ribosome Frequency ' + target)
    #ax.set_yscale('log')
    hist = plt.hist(pos_array, bins=range(0,gene_length + 1,5),density=False)
    plt.xlabel('Distance from AUG (codons)')
    plt.ylabel('Ribosome Frequency')
    plt.show()
    
    return

In [8]:
def increase(start, end, curr, rise, rate, default):
    run = abs(start - end)
    if run == 0:
        if curr == start:
            rate = rise
    else:
        slp = rise / run
        # increase from "0" to slider
        if curr >= start and curr <= end:
            if curr == end:
                rate = rise
            else:
                rate = ((slp * curr) + (-slp * start))
                if rate == 0:
                    rate = default

    return rate

In [9]:
def decrease(start, end, curr, fall, rate, default):
    run = abs(start - end)
    if run == 0:
        if curr == start:
            rate = default
    else:
        slp = fall / run
        # decrease from slider to "0"
        if curr >= start and curr <= end:
            if curr == end:
                rate = default
            else:
                rate = ((slp * curr) + (-slp * end))
                if rate == 0:
                    rate = default
    
    return rate

In [14]:
"""
Description: Determines whether or not the ribosomes on the simulated mRNA will collide, which ribosome is allowed to
    move, and updates the positions and number of timesteps present at a given location of all ribosomes corresponding
    to a simulated mRNA.
Inputs: ribosomes - the number of ribosomes to be run on a given mRNA
        time - the length of the simulation (sec.)
        ribo_size - the size of the ribosome object (nucleotides)
        probability - the probability that a ribosome will move forward from its current position
        kI - initiation rate
        kE - elongation rate
        n_mRNA - total number of mRNAs
Return: ribosome_list - list containing all ribosome objects corresponding to a given mRNA
"""
def simulate(ribosomes, time, ribo_size, n_mRNA, prob, kI, kE, t_st_init, t_st_elo,
               t_crit_st_init, t_crit_st_elo, t_stp_init, t_stp_elo, t_crit_stp_init,
             t_crit_stp_elo):
    global gene_length
    global probability
    global res
    global codon_size
    probability = prob
    dead = res ** -codon_size
    ribo_size = math.floor(ribo_size / codon_size)
    if kE == 0:
        slider_kE = dead
    else:
        slider_kE = kE
    kE = dead
    if kI == 0:
        slider_kI = dead
    else:
        slider_kI = kI
    kI = dead
    # list of user-defined number of ribosomes
    ribosome_list = [ribosome() for complex in range(ribosomes)]
    flag, gene_length, mRNA = read_gene(args.filename)
    # splits the simulation into 0.01sec. resolution
    discrete_time = [t for t in range(time * res)]
    for t_step in discrete_time:
        # Increase initiation rate starting at t_st_init until t_crit_init to kI
        init_st_run = t_crit_st_init - t_st_init
        if init_st_run == 0:
            if t_step == t_st_init:
                kI = slider_kI
        else:
            init_rise = slider_kI
            # calculate slope to adjust kI by for each t_step in range
            init_up_slp = init_rise / init_st_run
            # increase initiation from "0" to slider kI value
            if t_step >= t_st_init and t_step <= t_crit_st_init:
                if t_step == t_crit_st_init:
                    kI = slider_kI
                else:
                    kI = ((init_up_slp * (t_step)) + (-init_up_slp * t_st_init))
                    if kI == 0:
                        kI = dead

        # Decrease initiation rate starting at t_stp_init until t_crit_stp_init to 0
        init_fall = -slider_kI
        init_stp_run = t_crit_stp_init - t_stp_init
        if init_stp_run == 0:
            if t_step == t_stp_init:
                kI = dead
            else:
                # decrease initiation from slider kI to effectively 0
                if t_step <= t_stp_init and t_step >= t_crit_stp_init:
                    if t_step == t_crit_stp_init:
                        kI = dead
        else:
            init_dwn_slp = init_fall / init_stp_run
            # decrease initiation from slider kI to effectively 0
            if t_step <= t_stp_init and t_step >= t_crit_stp_init:
                if t_step == t_crit_stp_init:
                    kI = dead
                else:
                    kI = ((init_dwn_slp * (t_step)) + (-init_dwn_slp * t_crit_stp_init))
                    if kI == 0:
                        kI = dead
        
        # Increase elongation rate starting at t_st_elo until t_crit_elo to kE
        elo_st_run = t_crit_st_elo - t_st_elo
        elo_rise = slider_kE
        if elo_st_run == 0:
            if t_step == t_st_elo:
                kE = slider_kE
        else:
            elo_up_slp = elo_rise / elo_st_run
            # increase elongation from "0" to slider kE value
            if t_step >= t_st_elo and t_step <= t_crit_st_elo:
                if t_step == t_crit_st_elo:
                    kE = slider_kE
                else:
                    kE = ((elo_up_slp * (t_step)) + (-elo_up_slp * t_st_elo))
                    if kE == 0:
                        kE = dead
        
        # Decrease elongation rate starting at t_stp_init until t_crit_stp_init to 0
        elo_fall = -slider_kE
        elo_stp_run = t_crit_stp_elo - t_stp_elo
        if elo_stp_run == 0:
            if t_step == t_stp_elo:
                kE = dead
            else:
                # decrease elongation from slider kE to "0"
                if t_step <= t_stp_elo and t_step >= t_crit_stp_elo:
                    if t_step == t_crit_stp_elo:
                        kE = dead
        else:
            elo_dwn_slp = elo_fall / elo_stp_run
            # decrease elongation from slider kE to "0"
            if t_step <= t_stp_elo and t_step >= t_crit_stp_elo:
                if t_step == t_crit_stp_elo:
                    kE = dead
                else:
                    kE = ((elo_dwn_slp * (t_step)) + (-elo_dwn_slp * t_crit_stp_elo))
                    if kE == 0:
                        kE = dead
                        
        # each ribosome...
        for complex in range(len(ribosome_list)):
            # compared ribosome counter variable
            n_ribo = 0
            step = ribosome_list[complex].counter
            if t_step % int(res / kI) == 0:
                if step % int(res / kE) == 0:
                    for ribo in ribosome_list:
                        if ribosome_list.index(ribo) == complex:
                            n_ribo += 1
                        elif ribo.position > ribosome_list[complex].position:
                            if ribo.position - ribosome_list[complex].position > ribo_size: # can move
                                n_ribo += 1
                            else: 
                                break
                        elif ribo.position < ribosome_list[complex].position:
                            n_ribo += 1
                        elif ribo.position == -1:
                            n_ribo += 1
                        else:
                            n_ribo += 1
                    if n_ribo == len(ribosome_list):
                        ribosome_list[complex] = move(ribosome_list[complex])
                        if ribosome_list[complex].position > gene_length:
                            ribosome_list[complex].position = -1
                            ribosome_list[complex].counter = 1
                    else: ribosome_list[complex].counter += 1
                # if only initiation timestep
                else:
                    if ribosome_list[complex].position == -1:
                        for ribo in ribosome_list:
                            if ribosome_list.index(ribo) == complex:
                                n_ribo += 1
                            elif ribo.position > ribosome_list[complex].position:
                                if ribo.position - ribosome_list[complex].position > ribo_size: # can move
                                    n_ribo += 1
                                else: 
                                    break
                            elif ribo.position < ribosome_list[complex].position:
                                n_ribo += 1
                            elif ribo.position == -1:
                                n_ribo += 1
                            else:
                                n_ribo += 1
                        if n_ribo == len(ribosome_list):
                            ribosome_list[complex] = move(ribosome_list[complex])
                            if ribosome_list[complex].position > gene_length:
                                ribosome_list[complex].position = -1
                                ribosome_list[complex].counter = 1
                        else: ribosome_list[complex].counter += 1
                    else: ribosome_list[complex].counter += 1
            # if not initiation timestep
            else:
                # if elongation timestep but not initiation timestep
                if step % int(res / kE) == 0:
                    if ribosome_list[complex].position == -1:
                        ribosome_list[complex].counter += 1
                    else:
                        for ribo in ribosome_list:
                            if ribosome_list.index(ribo) == complex:
                                n_ribo += 1
                            elif ribo.position > ribosome_list[complex].position:
                                if ribo.position - ribosome_list[complex].position > ribo_size: # can move
                                    n_ribo += 1
                                else: 
                                    break
                            elif ribo.position < ribosome_list[complex].position:
                                n_ribo += 1
                            elif ribo.position == -1:
                                n_ribo += 1
                            else:
                                n_ribo += 1
                        if n_ribo == len(ribosome_list):
                            ribosome_list[complex] = move(ribosome_list[complex])
                            if ribosome_list[complex].position > gene_length:
                                ribosome_list[complex].position = -1
                                ribosome_list[complex].counter = 1
                        else: ribosome_list[complex].counter += 1
                # if not initiation nor elongation timestep
                else:
                    ribosome_list[complex].counter += 1
    return ribosome_list

In [11]:
"""
Description: Creates a list of lists containing
Inputs: ribosomes - the number of ribosomes to be run on a given mRNA
        time - the length of the simulation (sec.)
        ribo_size - the size of the ribosome object (nucleotides)
        probability - the probability that a ribosome will move forward from its current position
        kI - initiation rate
        kE - elongation rate
        n_mRNA - total number of mRNAs
Return: None
"""
def concatenate(ribosomes, time, ribo_size, n_mRNA, prob, kI, kE, t_st_init, t_st_elo,
               t_crit_st_init, t_crit_st_elo, t_stp_init, t_stp_elo, t_crit_stp_init, t_crit_stp_elo):
    global gene_length
    occupancy = []
    
    for mRNAs in range(n_mRNA):
        #occupancy.append(simulate(ribosomes,time,ribo_size,prob,kI,kE,t_init,t_elo,t_init_st,t_elo_st))
        occupancy.append(simulate(ribosomes, time,
                                  ribo_size, n_mRNA,
                                  prob, kI,
                                  kE, t_st_init,
                                  t_st_elo, t_crit_st_init,
                                  t_crit_st_elo, t_stp_init,
                                  t_stp_elo, t_crit_stp_init,
                                  t_crit_stp_elo))
    create_histogram(occupancy)
    print("gene length: {} codons".format(gene_length))
    
    return

In [15]:
"""
Initialize the sliders that control length of simulation, number of ribosomes, number of mRNA, size of mRNAs, kI, kE,
    and the probability of moving to the next codon. Update the histogram when Run Interact button is pressed.
"""
initiation = widgets.IntSlider(min = 0, max = 10, value = 1, step = 1, description = 'kI - try/sec.');
init_stop_time = widgets.IntSlider(min = 0, max = 90001, value = 50, step = 1, description = 'sec. to kI = 0');
elongation = widgets.IntSlider(min = 0, max = 20, value = 20, step = 1, description = 'kE - try/sec.');
elo_stop_time = widgets.IntSlider(min = 0, max = 90001, value = 50, step = 1, description = 'sec. to kE = 0');
init_start_time = widgets.IntSlider(min = 0, max = 90001, value = 50, step = 1, description = 'sec. delay kI');
elo_start_time = widgets.IntSlider(min = 0, max = 90001, value = 50, step = 1, description = 'sec. delay kE');
init_crit_inc_time = widgets.IntSlider(min = 0, max = 90001, value = 50, step = 1, description = 'sec. until kI');
elo_crit_inc_time = widgets.IntSlider(min = 0, max = 90001, value = 50, step = 1, description = 'sec. until kE');
prob_move = widgets.FloatSlider(min = 0.00, max = 1.00, value = 0.50, step = 0.01, description = 'P(forward)');
sizes = widgets.IntSlider(min = 25, max = 35, value = 30, step = 1, description = 'Ribo size (nt.)');
secs = widgets.IntSlider(min = 0, max = 90000, value = 100, step = 10, description = 'time (sec.)');
ribo = widgets.IntSlider(min = 0, max = 25, value = 25, step = 1, description = 'n(ribosomes)');
num_cules = widgets.IntSlider(min = 0, max = 5000, value = 10, step = 1, description = 'n(mRNA)');
init_decay_time = widgets.IntSlider(min = 0, max = 90001, value = 20, step = 1, description = 'kI start decay');
elo_decay_time = widgets.IntSlider(min = 0, max = 90001, value = 20, step = 1, description = 'kE start decay');
interact_manual(concatenate,
                ribosomes = ribo,
                time = secs,
                ribo_size = sizes,
                n_mRNA = num_cules,
                prob = prob_move,
                kI = initiation,
                kE = elongation,
                t_st_init = init_start_time,
                t_st_elo = elo_start_time,
                t_crit_st_init = init_crit_inc_time,
                t_crit_st_elo = elo_crit_inc_time,
                t_stp_init = init_decay_time,
                t_stp_elo = elo_decay_time,
                t_crit_stp_init = init_stop_time,
                t_crit_stp_elo = elo_stop_time);

interactive(children=(IntSlider(value=25, description='n(ribosomes)', max=25), IntSlider(value=100, description='time (sec.)', max=90000, step=10), IntSlider(value=30, description='Ribo size (nt.)', max=35, min=25), IntSlider(value=10, description='n(mRNA)', max=5000), FloatSlider(value=0.5, description='P(forward)', max=1.0, step=0.01), IntSlider(value=1, description='kI - try/sec.', max=10), IntSlider(value=20, description='kE - try/sec.', max=20), IntSlider(value=50, description='sec. delay kI', max=90001), IntSlider(value=50, description='sec. delay kE', max=90001), IntSlider(value=50, description='sec. until kI', max=90001), IntSlider(value=50, description='sec. until kE', max=90001), IntSlider(value=20, description='kI start decay', max=90001), IntSlider(value=20, description='kE start decay', max=90001), IntSlider(value=50, description='sec. to kI = 0', max=90001), IntSlider(value=50, description='sec. to kE = 0', max=90001), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))